# RWKV Token Shift Experiment E
This model is a custom model containing
- 24 layers
- 4096 embedding size
- 5.6B params

This scales up the memory model, to be closer in embedding size to the 7B / 14B models
While having the reduced layer count, to keep the param count resonable.

This gives a rather realistic approximate memory benchmark test for the 7B / 14B models

Due to the size of the model, it is required to run on at minimum 4 x 40G GPU instances

**Note:** This project assumes you have the rwkv-infctx conda env setup

---

```bash
# ninja-build is required for the new trainer
sudo apt-get install ninja-build

# Update conda & its package listings
conda update conda

# Virtual env, with python 3.10
# python 3.11 may have issues with torch.compile / h100s
conda create -n rwkv-infctx python=3.11 pip
conda activate rwkv-infctx

# Install pytorch (>=2.0.1)
conda install -y pytorch==2.0.1 torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

# Verify your pytorch version 
python -c "import torch; print(torch.__version__)"

# We use python -m pip, instead of pip directly, as it resolve issues with venv not loading the right pip
python -m pip install datasets transformers 
python -m pip install lightning==2.0.5 deepspeed==0.10.0
python -m pip install ninja numexpr jsonargparse 'jsonargparse[signatures]'
python -m pip install lm-dataformat ftfy sentencepiece tokenizers wandb

# Additionally, if we are intending to run the jupyter notebook CLI
python -m pip install jupyter nbclient

# if so, we can run this notebook via the command line with
# jupyter nbconvert --execute --to notebook --inplace TokenShift-E-basemodel.ipynb
```
---

# Basic Setup

In [ ]:
# Setup the various required folders
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

# Intialize the model, if the file does not exist
!cd ../../../../RWKV-v4wavenet/ && python3 \
    ./init_model.py \
    --n_layer 24 --n_embd 4096 --vocab_size neox \
    --skip-if-exists \
    ../model/L24-D4096-init.pth

# List the model file (and its size)
!ls -alh ../../../../model/L24-D4096-init.pth

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="TokenShift-E"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4wavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

In [ ]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-E-enwiki.yaml"

## Stage 1 : Foundation model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-E-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki Foundation (data-ctx=8192, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.ctx_len=8192 \
        --model.bptt_learning_range=1 \
        --auto-resume-ckpt-dir "auto"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/TokenShift-E-enwiki/last.ckpt" "../model/TokenShift-E-Stage1.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/TokenShift-E-Stage1.pth"

In [ ]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py ../model/TokenShift-E-Stage1.pth "cuda fp32"

In [ ]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-E-Stage1.pth"

# Stage 2 : Instruct Tuning

In [ ]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/TokenShift-E-instruct.yaml"

In [ ]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/TokenShift-E-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1 \
        --auto-resume-ckpt-dir "auto"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/TokenShift-E-instruct/last.ckpt" "../model/TokenShift-E-Stage2.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/TokenShift-E-Stage2.pth"

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/TokenShift-E-Stage2.pth" "cuda fp32"

In [ ]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_model_memory_guided.py "{PROJECT_DIR}/model/TokenShift-E-Stage2.pth"